In [1]:
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install matplotlib


import sys
import os

sys.path.append(os.path.abspath('../src'))



import numpy as np
import pandas as pd
from neural_network.layers.dense_layer import DenseLayer
from neural_network.layers.dropout_layer import DropoutLayer
from neural_network.optimizer import GradientDescent
from neural_network.losses import CrossEntropyLoss,MeanSquaredError
from neural_network.utils import normalize_data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import ( accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report, mean_absolute_error,
    mean_squared_error, r2_score
)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:


df = pd.read_csv('../data/regression/student-por.csv')

categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {col: LabelEncoder() for col in categorical_cols}
for col in categorical_cols:
    df[col] = label_encoders[col].fit_transform(df[col])

df['absences'] = df['absences'].clip(upper=20)

scaler = StandardScaler()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

X = df.drop(columns=['G3'])
y = df['G3']


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
class NeuralNetwork:
    def __init__(self, layers, loss, optimizer):
        self.layers = layers
        self.loss = loss
        self.optimizer = optimizer
        
    def forward(self, X):
        output = X
        for layer in self.layers:
            output = layer.forward(output)
        return output
    
    def backward(self, y_true, y_pred):
        grad = self.loss.backward(y_true, y_pred)
        for layer in reversed(self.layers):
            grad = layer.backward(grad)
    
    def update(self):
        for layer in self.layers:
            params = layer.get_parameters()
            if params:
                grads = layer.get_gradients()
                weights_updated, biases_updated = self.optimizer.update(
                    params['weights'], params['biases'],
                    grads['weights'], grads['biases']
                )
                layer.set_parameters({'weights': weights_updated, 'biases': biases_updated})


In [139]:
input_size = X_train.shape[1]

layers = [
    DenseLayer(input_size=input_size, output_size=128, activation='relu'), 
    DropoutLayer(rate=0.2), 
    DenseLayer(input_size=128, output_size=64, activation='relu'),  
    DenseLayer(input_size=64, output_size=32, activation='linear'), 

    DenseLayer(input_size=32, output_size=1, activation='linear')  
]

loss = MeanSquaredError()

optimizer = GradientDescent(learning_rate=0.01)  
model = NeuralNetwork(layers=layers, loss=loss, optimizer=optimizer)


In [140]:

epoch = 0
previous_loss = float('inf')  
loss_value = 0.5  
convergence_threshold = 0.05 

while loss_value > convergence_threshold:
    epoch += 1
    
    predictions = model.forward(X_train)
    
    loss_value = loss.calculate(y_train.values.reshape(-1, 1), predictions)
    
    model.backward(y_train.values.reshape(-1, 1), predictions)
    
    model.update()
    
    if epoch % 10 == 0:
        mean_absolute_error = np.mean(np.abs(y_train.values - predictions.flatten()))
        print(f'Epoch {epoch:04d} | Loss: {loss_value:.4f} | MAE: {mean_absolute_error:.4f}')
    
    if abs(previous_loss - loss_value) < 1e-6:
        print("Converged: Loss improvement below threshold.")
        break
    
    previous_loss = loss_value


Epoch 0010 | Loss: 1.1355 | MAE: 0.8020
Epoch 0020 | Loss: 1.0703 | MAE: 0.7774
Epoch 0030 | Loss: 1.0228 | MAE: 0.7594
Epoch 0040 | Loss: 0.9841 | MAE: 0.7444
Epoch 0050 | Loss: 0.9509 | MAE: 0.7313
Epoch 0060 | Loss: 0.9219 | MAE: 0.7195
Epoch 0070 | Loss: 0.8956 | MAE: 0.7086
Epoch 0080 | Loss: 0.8708 | MAE: 0.6984
Epoch 0090 | Loss: 0.8473 | MAE: 0.6889
Epoch 0100 | Loss: 0.8248 | MAE: 0.6797
Epoch 0110 | Loss: 0.8028 | MAE: 0.6705
Epoch 0120 | Loss: 0.7817 | MAE: 0.6617
Epoch 0130 | Loss: 0.7613 | MAE: 0.6530
Epoch 0140 | Loss: 0.7417 | MAE: 0.6448
Epoch 0150 | Loss: 0.7225 | MAE: 0.6367
Epoch 0160 | Loss: 0.7039 | MAE: 0.6288
Epoch 0170 | Loss: 0.6860 | MAE: 0.6210
Epoch 0180 | Loss: 0.6686 | MAE: 0.6132
Epoch 0190 | Loss: 0.6513 | MAE: 0.6054
Epoch 0200 | Loss: 0.6348 | MAE: 0.5976
Epoch 0210 | Loss: 0.6187 | MAE: 0.5897
Epoch 0220 | Loss: 0.6030 | MAE: 0.5818
Epoch 0230 | Loss: 0.5879 | MAE: 0.5741
Epoch 0240 | Loss: 0.5734 | MAE: 0.5665
Epoch 0250 | Loss: 0.5589 | MAE: 0.5589


In [142]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred_test = model.forward(X_test)

test_loss = loss.calculate(y_test.values.reshape(-1, 1), y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test.flatten())  
mse = mean_squared_error(y_test.values, y_pred_test.flatten())
rmse = np.sqrt(mse)

print(f"Test Loss: {test_loss:.4f}, MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}")


Test Loss: 0.7864, MAE: 0.6867, MSE: 0.7864, RMSE: 0.8868
